# Bioinformatics Workshop 1

Welcome to the first workshop of the MBSI Bioinformatics course! In this Jupyter notebook, we will introduce you to the basics of genetic code manipulation with scikit-bio. Feel free to work through the exercises at your own pace, and ask the tutors if you encounter any problems. You can run the coding cells using Shift + Enter.

---

## Getting started with biopython

The first thing we will have to do is import Biopython, along with other relevant libraries.

In [1]:
#Import libraries 
import numpy as np
import pandas as pd
import Bio
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import GC
%cd "C:\Users\vince\Documents\mbsi"

C:\Users\vince\Documents\mbsi


Next, we will use Biopython's `seq` object to input our first snippets of DNA code. Remember that only characters in the IUPAC DNA character set are supported, although in our case we will simply use A, T, C and G. The characters should be input as a string with no spaces, eg. `'ATGAGGCCT'`. Have a go in the cell below, and check out the result!

In [37]:
#Exercise 0a
Seq('ATGAGGCCT')

Seq('ATGAGGCCT')

Sequences can be treated as normal python strings. Thus, we can get the length and iterate over the elements. For our first real exercise, write a function that iterates over a given sequence and prints the index next to the letter.

In [38]:
#Exercise 0b
def printbases(seq):
    for index, letter in enumerate(seq):
        print("%i %s" % (index, letter))

In [39]:
#Test your code here
printbases(Seq('ATTACG'))

0 A
1 T
2 T
3 A
4 C
5 G


#### SeqRecord objects

The SeqRecord is a class that allows for further metadata to be associated with a sequence, along with being the data type used for the `seqIO` input/output interface used by Biopython. It contains the following data as attributes, as defined by the biopython cookbook: (http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec34)

- .seq
    - The sequence itself, typically a Seq object.
- .id
    - The primary ID used to identify the sequence – a string. In most cases this is something like an accession number.
- .name
    - A “common” name/id for the sequence – a string. In some cases this will be the same as the accession number, but it could also be a clone name. I think of this as being analogous to the LOCUS id in a GenBank record.
- .description
    - A human readable description or expressive name for the sequence – a string.
- .letter_annotations
    - Holds per-letter-annotations using a (restricted) dictionary of additional information about the letters in the sequence. The keys are the name of the information, and the information is contained in the value as a Python sequence (i.e. a list, tuple or string) with the same length as the sequence itself. This is often used for quality scores (e.g. Section ‍20.1.6) or secondary structure information (e.g. from Stockholm/PFAM alignment files).
- .annotations
    - A dictionary of additional information about the sequence. The keys are the name of the information, and the information is contained in the value. This allows the addition of more “unstructured” information to the sequence.
- .features
    - A list of SeqFeature objects with more structured information about the features on a sequence (e.g. position of genes on a genome, or domains on a protein sequence). The structure of sequence features is described below in Section ‍4.3.
- .dbxrefs
    - A list of database cross-references as strings.

To start off, you can create a SeqRecord  with just a Seq object.

In [47]:
my_seq = Seq('ATTAG')
my_seqr = SeqRecord(my_seq)

Next, you can use the Seqrecord attributes to modify the name, id, description and other information.

In [49]:
my_seqr.id = 'MX123812'
my_seqr.name = 'Cool name'
my_seqr.description = 'Just a sequence I made up'
print(my_seqr)

ID: MX123812
Name: Cool name
Description: Just a sequence I made up
Number of features: 0
Seq('ATTAG')


Next, we will import real-world data using Seqrecord.

#### Importing Fasta and Fastq files

Today, we'll be using Genbank, Fasta and Fastq files containing real-world genetic information. To import them, we will use Entrez to retrieve data from NCBI's databases such as PubMed, GenBank and others. In this case, we first use `Entrez.email` to give an email to NCBI to let them know who we are. Next, we use `Entrez.efetch` to retrieve the desired information. Here, it is a fasta file from the nucleotide database, with the id of NC_045512.2. Retmode denotes the mode of retrieval, in this case trext instead of xml. Next, `SeqIO.read()` is used to read the fasta file retrieved into a Seqrecord object.

In [3]:
import Bio
from Bio import Entrez, SeqIO
Entrez.email = 'vincentkoh1998@gmail.com'
handle = Entrez.efetch(db="nucleotide", id='NC_045512.2', rettype = 'gb', retmode="text")
record = SeqIO.read(handle,'gb')
print(record)

ID: NC_045512.2
Name: NC_045512
Description: Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome
Database cross-references: BioProject:PRJNA485481
Number of features: 57
/molecule_type=ss-RNA
/topology=linear
/data_file_division=VRL
/date=18-JUL-2020
/accessions=['NC_045512']
/sequence_version=2
/keywords=['RefSeq']
/source=Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)
/organism=Severe acute respiratory syndrome coronavirus 2
/taxonomy=['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']
/references=[Reference(title='A new coronavirus associated with human respiratory disease in China', ...), Reference(title='Programmed ribosomal frameshifting in decoding the SARS-CoV genome', ...), Reference(title='The structure of a rigorously conserved RNA element within the SARS virus genome', ...), Reference(title="A phyl

Wow- now _that_ is a long sequence of DNA. But just how long? We can use `len()` as usual on a  Seq data type to find out. 

In [6]:
len(record.seq)

29903

Yeah, that's pretty long. Luckily, `scikit-bio` has a variety of useful methods we can use on the DNA data type to help us manage and interpret the sequence. Firstly, qualitative methods such as `gaps()` and `has_metadata()` help us quickly make sense of the seqeunce.

## Exercise 1: Counting Nucleotides

For our first exercise, we will write a simple function that will count the number of times `A`, `T`, `C` and `G` appear in our DNA sequence. To help you with this, here is a scaffold for the code needed:


- Create four **variables** that will keep track of the nucleotide counts. 
- Create a `for` loop to iterate along the enitre seqeunce
- Inside that `for` loop, add 1 to the counting variables everytime the iterator encounters the variables' corresponding letter.
- Return a dictionary of the nucleotide letter and its corresponding count.

> 💡 **Hint:**
Convert the sequence into a string before or in the for loop.

In [9]:
#Excercise 1a
def NuCounter(Sequence):
    #your code here
    countA = 0
    countT = 0
    countC = 0
    countG = 0
    for nuc in str(Sequence):
        if nuc == 'A':
            countA += 1
        elif nuc == 'T':
            countT += 1
        elif nuc == 'C':
            countC += 1
        elif nuc == 'G':
            countG += 1
        
    return {'A':countA, 'T':countT, 'C':countC, 'G':countG}


In [10]:
#Test your code below
print(NuCounter(Seq('ATCGGTCCAAGTACAG')) )#Should return {'A': 5, 'T': 3, 'C': 4, 'G': 4}
print(NuCounter(record.seq))

{'A': 5, 'T': 3, 'C': 4, 'G': 4}
{'A': 8954, 'T': 9594, 'C': 5492, 'G': 5863}


Good job! As it turns out, `Biopython` already has a built-in method to count the nucleotides, `count()`. Oh well, at least we know how that works now. Using the `count` method below:

In [7]:
print(record.seq.count('A'))
print(record.seq.count('T'))
print(record.seq.count('C'))
print(record.seq.count('G'))


8954
9594
5492
5863


Good to see that our results match up. Looks like our function still saves us a bit of work since we only need to input the sequence once! In addition to the A,T,C and Gs, we can also ahve other nucleotides pop up, such as N. This just means that the specific position can be occupied by any nucleotide. There are other nucleotide codes such as B, which signifies if a position is occupied by a C, G or T. Try modifying your code in exercise 1a to include counting the N positions for the following sample:

In [31]:
Sample = SeqIO.read('Sample.fasta', 'fasta')
print(Sample)

ID: s_harrisii_sample
Name: s_harrisii_sample
Description: s_harrisii_sample
Number of features: 0
Seq('TATTTACTTTCAGTTTTATTAATCTCCTCATTTTATTGGAAGAATTTTTGTTTG...TTT')


In [22]:
#Excercise 1b
def NuCounterN(Sequence):
    #your code here
    countA = 0
    countT = 0
    countC = 0
    countG = 0
    countN = 0
    for nuc in str(Sequence):
        if nuc == 'A':
            countA += 1
        elif nuc == 'T':
            countT += 1
        elif nuc == 'C':
            countC += 1
        elif nuc == 'G':
            countG += 1
        else:
            countN += 1
        
    return {'A':countA, 'T':countT, 'C':countC, 'G':countG, 'N':countN}


In [32]:
#Test your code below
print(NuCounterN(Seq('ATNCGGTCCAAGNNTACAG')) )#Should return {'A': 5, 'T': 3, 'C': 4, 'G': 4, 'N': 3}
print(NuCounterN(Sample.seq))

{'A': 5, 'T': 3, 'C': 4, 'G': 4, 'N': 3}
{'A': 42333, 'T': 44167, 'C': 22201, 'G': 21760, 'N': 103}


**Great!** Another good use for counting nucleotides is determining the _GC content of a seqeunce_. The GC content is the percentage of bases in DNA or RNA that are either guanine (G) or cytosine (C). Knowing the GC content of a sequence has a variety of biological uses. For example, determining the evolution of a genome by recombination through tracking the GC-content over time, and discovering relationships between chromosome sizes and life-history traits.

For us to get to that point though, we first have to write a function to determine the GC content. You can adapt the code you have already written in the previous exercises to make your life eaiser. Return the output as the sentence `'This sequence has a GC content of [your result]%.'`. 

In [54]:
#Exercise 1c
def GcCount(Sequence):
    #Your code here
    countG = 0
    countC = 0
    for nuc in str(Sequence):
        if nuc == 'G':
            countG += 1
        elif nuc == 'C':
            countC += 1
    
    GCpercent = ((countG+countC)/len(Sequence))*100
    
    return 'This sequence has a GC content of ' + str(GCpercent) + '%.'
    

In [55]:
#Test your code below
print(GcCount(record.seq)) #output should be 'This sequence has a GC content of 33.670077509880215%.'

This sequence has a GC content of 37.97277865097147%.


**Excellent!** Again, `Biopython` has us beat. The in-built function `GC()` and calculates the GC content for us already.

In [33]:
print(GC(record.seq)) #Note that the output is a decimal, not percentage.

37.97277865097148


Now that you are familiar with counting nucleotides on one sequence, time to do it on a list of sequences in a Fastq file.

---

## Exercise 2: Working with multiple sequences

As mentioned in the slides, the FASTQ file differs from the usual FASTA format in that it contains a  _set_ of reads, each with their own quality scores and metadata, rather than just a single string. This means that a slightly different method of importing and code is needed.

#### Importing a set of reads

Firstly, we shall use `SeqIO.parse` to read in the FASTQ file. This time, we will initialise a list for the reads, then append the reads into the list. Note that we need two arguments to input into `SeqIO.parse`: the file name and the file type. Here, we have used 'fastq-sanger' to indicate that the quality scores are in base 33 ASCII, rather than the base 64.

In [17]:
#Append to a list of reads
Readset = []
for seq_record in SeqIO.parse("ws1.fastq", "fastq-sanger"):
    Readset.append(seq_record)


In [18]:
#Lets see what we're working with:
print(len(Readset))
#print(Readset)

200


Again, that's a whole lot of reads. So how are we going to make sense of them? Well, getting the minimum, maximum and average lengths of the reads would be a good start. Here is a scaffold to use when constructing the function:

- Create an empty list to store the read lengths of all the sequences in the readset.
- Iterate using a for loop across all the reads in the readset, and store the lengths of each read in the list.
- Calculate the minimum, maximum and average lengths using the `min()`, `max()` and `sum()` functions.
- Return a dictionary containing the labels `min_length`, `max_length` and `avg_length`.

In [14]:
#Exercise 2a
def read_lengths(readset):
    #Your code here
    
    read_lengths = []
    for read in readset:
        read_lengths.append(len(read))
    
    lengthmin = min(read_lengths)
    lengthmax = max(read_lengths)
    lengthavg = sum(read_lengths)/float(len(read_lengths))
    
    return {'min_length':lengthmin, 'max_length':lengthmax, 'avg_length':lengthavg}

In [19]:
#Test your code here
print(read_lengths(Readset)) #Output should be "{'min_length': 56, 'max_length': 76, 'avg_length': 75.445}"

{'min_length': 56, 'max_length': 76, 'avg_length': 75.445}


**Nice!** Now, let's put together what we have learned so far. Create a function that outputs the average number of nucleotides A, T, C and G in a given readset. You do not have to consider any other DNA code character. This time, we will be using nested for loops, the outer one iterating over the reads, and the inner one iterating over each nucleotide in the sequence.

Here is a scaffold to help you along:
- Initialise four lists to keep track of the average counts of each nucleotide for each read
- Start a for loop interating over every read in the readset
    - **inside this loop:**
    - Create four **variables** that will keep track of the nucleotide counts. 
    - Create a `for` loop to iterate along the enitre seqeunce
        -  **Inside this loop:** 
        - Add 1 to the counting variables everytime the iterator encounters the variables' corresponding letter.
    - Append the counts to their corresponding lists.
- Take the average of each list (you can use `np.mean()` instead of using `sum()` and dividing)
- Return a dictionary containg the labels `avgA`, `avgT`, `avgC`, and `avgG` with their corresponding average counts.

> 💡 **Hint:**
A lot of the code inside the first `for` loop can be adapted or directly taken from your function in Exercise 1a!


In [34]:
#Exercise 2b
def avgATCG(readset):
    #Your code here
    
    listA = []
    listT = []
    listC = []
    listG = []
    
    for read in readset:
        countA = 0
        countT = 0
        countC = 0
        countG = 0
        
        for nuc in str(read):
            if nuc == 'A':
                countA += 1
            elif nuc == 'T':
                countT += 1
            elif nuc == 'C':
                countC += 1
            elif nuc == 'G':
                countG += 1
            
        listA.append(countA)
        listT.append(countT)
        listC.append(countC)
        listG.append(countG)
    
    return {'avgA':np.mean(listA), 'avgT':np.mean(listT), 'avgC':np.mean(listC), 'avgG':np.mean(listG)}

In [35]:
#Test your code here
print(avgATCG(Readset)) #output should be '{'avgA': 19.66, 'avgT': 18.905, 'avgC': 18.31, 'avgG': 18.57}'

{'avgA': 15.065, 'avgT': 14.32, 'avgC': 13.665, 'avgG': 13.945}


**Well Done!** Nested `for` loops are essential when dealing with a _set_ of sequences or reads. Now, we shall now take a look at transcribing, complementing and translating DNA sequences.

---

## Exercise 3: Transcribing and Translating


Genes provide information to create _proteins_. The production happens through two processes: **Transcription** and **Translation**. Transcription involves the using the DNA strand as a template to build a sequence of _RNA_. During translation, amino acids are produced according to the information from the RNA sequence. In the next few exercises, we will create a series of functions that will simulate this process.

Firstly, let's understand the basics of transcription. 
- **The four nucleotides found in DNA:** Adenine (A), Cytosine (C), Guanine (G), and Thymine (T).
- **The four nucleotides found in RNA**: Adenine (A), Cytosine (C), Guanine (G), and Uracil (U).

As you can see, RNA does not have a T nucleotide- instead, it contains Uracil(U) which we can treat as an equivalent.

For each DNA strand, the transcribed RNA strand is made by adding on the DNA's _complement nucleotide_. In this case:
- A &rarr; U
- T &rarr; A
- C &rarr; G
- G &rarr; C

In the exercise below, we shall be essentially following this process to output a transcribed sequence of RNA form a DNA input.

Here is a useful scaffold for the code needed:
- Create a `for` loop iterating over every nucleotide of the sequence
- Use `if` statements to replace the nucleotide at that specific position with it's RNA complement
- Return a sequence of the transcribed RNA

In [36]:
#Exercise 3a
def transcribe(Sequence):
    #Your code here
    l = list(str(Sequence))
    
    for i in range(len(Sequence)):
      
      if(l[i] == 'G'):
        l[i] = 'C'
  
      elif(l[i] == 'C'):
          l[i] = 'G'
  
      elif (l[i] == 'T'):
          l[i] = 'A'
  
      elif (l[i] == 'A'):
          l[i] = 'U'
     
    output_seq = ''
    
    for base in l:
        output_seq += base
    
    return output_seq
        
    

In [37]:
#Test your code here
print(transcribe(Seq('GCTAA'))) #should return CGAUU
#print(transcribe(record.seq)) 

CGAUU


The next step is for the information in RNA to be **translated** to proteins. This is done by every three bases in the RNA sequence being correlated to an animo acid. For example, the three bases, or **codon**, 'GTC' codes for the amino acid valine. The sequences of animo acids form the building blocks of proteins, each with their own particualr functions. 

The coding system for the codons and amino acids is known as the genetic code. There are many different genetic codes, with the most common being the standard genetic code. This code can be called from skbio.

In [38]:
from Bio.Data import CodonTable
standard_table = CodonTable.unambiguous_dna_by_id[1]
print(standard_table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

Reading down the columns, the amino acid 'F', or phenylalanine, is coded by codons UUU and UUC, and so on. In the exercise below, we shall write a function that takes transcibed RNA seqeunces and outputs a string of its corresponding amino acids. This can be done by using a whole bunch of `if` and `elif` statements, but an easier way would be to use a dictionary, like the one provided:

In [39]:
stdcodeRNA = {
        'AUA':'I', 'AUC':'I', 'AUU':'I', 'AUG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACU':'T',
        'AAC':'N', 'AAU':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGU':'S', 'AGA':'R', 'AGG':'R',                 
        'CUA':'L', 'CUC':'L', 'CUG':'L', 'CUU':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCU':'P',
        'CAC':'H', 'CAU':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGU':'R',
        'GUA':'V', 'GUC':'V', 'GUG':'V', 'GUU':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCU':'A',
        'GAC':'D', 'GAU':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGU':'G',
        'UCA':'S', 'UCC':'S', 'UCG':'S', 'UCU':'S',
        'UUC':'F', 'UUU':'F', 'UUA':'L', 'UUG':'L',
        'UAC':'Y', 'UAU':'Y', 'UAA':'_', 'UAG':'_',
        'UGC':'C', 'UGU':'C', 'UGA':'_', 'UGG':'W',
    }

The function will now need two inputs - the sequence and its corresponding genetic code dictionary. A possible scaffold for the function would be as follows:

- Intitialise the output protein string
- Create a `for` loop iterating over every _three_ bases in the sequence
- **Inside this loop**:
    - Define a codon as a sequence from the current index to three indexes further down the sequence
    - Append the output protein string with the corresponding value in the dictionary using the codon as the key
- Return the string of proteins as the output

> 💡 **Hint:**
Ignore bases or codons containing bases other than A,U,C,G.

In [40]:
#Exercise 3b
def translate(Sequence, Gencode):
    #Your code here
    output_prot = ''
    
    for i in range(0, len(Sequence), 3):
        codon = Sequence[i:i+3]
        for key in Gencode:
            if codon == key:
                output_prot += Gencode[codon]
        
    return output_prot

In [42]:
#Test your code here
translate('UUCAUUCCA',stdcodeRNA) #Should return FIP
#print(translate(transcribe(record.seq),stdcodeRNA))

'FIP'

Great! Now, we can try combining our two functions into one that iterates over a `readset`. You can just call your functions that you have written inside this larger function to save yourself some work. Output a list of protein strings.


In [43]:
#Exercise 3c
def Readsetprot(readset, Gencode):
    
    def transcribe(Sequence):
        #Your code here
        l = list(str(Sequence))
    
        for i in range(len(Sequence)):
      
          if(l[i] == 'G'):
            l[i] = 'C'
  
          elif(l[i] == 'C'):
              l[i] = 'G'
  
          elif (l[i] == 'T'):
              l[i] = 'A'
  
          elif (l[i] == 'A'):
              l[i] = 'U'
     
        output_seq = ''
    
        for base in l:
            output_seq += base
    
        return output_seq
    
    def translate(Sequence, Gencode):
        #Your code here
        output_prot = ''
    
        for i in range(0, len(Sequence), 3):
            codon = Sequence[i:i+3]
            for key in Gencode:
                if codon == key:
                    output_prot += Gencode[codon]
        
        return output_prot
    
    #Your code here
    output_prot = []
    
    for read in readset:
        RNA = transcribe(read)
        output_prot.append(translate(RNA, Gencode))
        
    return output_prot
        

In [44]:
#Test your code here
Readsetprot(Readset,stdcodeRNA)

['PQPGQQSKQT',
 'AGHTDTADQ',
 'SQPPKGHAPTR',
 'NNTAANAQ',
 'RAKREKG',
 'KANQAGS',
 'QAPTTKRE',
 'GRGTAP',
 'RQTDEQKSTRP',
 'RKEHTP',
 'KRKRAKRA',
 'ETSN',
 'TSGAAKKK',
 'GGAGKTPRPP',
 'ARENRRQT',
 'GKTAQR',
 'ADEP',
 'GRRNRHSQS',
 'PRKQQHA',
 'TQGEQK',
 'ARPQSTR',
 'KAKKHD',
 'RPQNRGAQ',
 'QTGEQKAN',
 'EKG',
 'RATQEKKKKKKEKTR',
 'RTRGKKKKKKKT',
 'DQHQET',
 'KGKGE',
 'RKGP',
 'GERTA',
 'QAAGG',
 'PASPARGGRNES',
 'TEARQST',
 'ADGP',
 'KPRATQTQKN',
 'KDRH',
 'TDRPATP',
 'PGKRKAG',
 'RQPNE',
 'SRRREP',
 'KGGN',
 'AAATPDENKKPN',
 'RRPPTAAGGAQA',
 'PAATKAQTPT',
 'HKKKKRG',
 'TKGKAQGQ',
 'RHTGRQETS',
 'KRRQHAP',
 'RGNTRGRPG',
 'ANEPETKGRGKR',
 'NGP',
 'QEEREQT',
 'AASTSR',
 'GQKPKG',
 'SQGAGE',
 'SKAEQHKT',
 'PAQREGG',
 'PKTTSQE',
 'PQKQHRET',
 'ETPQNT',
 'RSAAAGG',
 'STKEGD',
 'GPAGPPTQT',
 'RATSKR',
 'PHQDAD',
 'PTPERQG',
 'TQTTTK',
 'RHQTRAAST',
 'RPGKDQS',
 'EA',
 'RPAQGPR',
 'GPTAQPTQE',
 'RHRGPGKKKTQ',
 'PRTGPPDPDA',
 'AGSKKATGKG',
 'ASKRAHGHK',
 'KASKQRGT',
 'PTPQQETGDR',
 'PTGKA',
 'G

Great! Now on to our last section of the workshop!

---

## Exercise 4: Motifs 

Motifs are repeating patterns of bases in a seqeunce with a well-defined function. Such motifs include DNA seqeunces in enhancer regions, adapter seqeunces for gene sequencing and RNA seqeunces like splicing signals. In this exercise, we will have a look at first counting and locating known DNA motifs, and also modifying a sequence by trimming unwanted motifs.


For our first exercise, create a function that takes two seq objects, the sequence and motif, and returns the count and a list of all locations by the index of the sequence in a tuple.

Here is an example scaffold for your code:
- Initialise a list of locations and variables for storing the current base, location and motif count
- Initialise a `while` loop while the current base is _less than_ the length of the input sequence
    - **Inside this while loop**:
    - Use the `sequence.find()` method to look for motifs starting at the current base
    - If the finder discovers a match for the motif, add 1 to the current base and count, and append the the locations list with the current base
    - If no motifs are found, increase the current base by 1
- Output a list of the count and the location list

In [8]:
#Exercise 4.1
def findmotifs(sequence, motif):
    #Your code here
    locations = []
    base = 0
    loc = 0
    count = 0

    while (base < len(sequence)):
        loc = sequence.find(motif, start=base)
        if (loc > 0):
            base = loc + 1
            locations.append(loc + 1)# adding 1 as result is 1-indexed
            count += 1
        else:
            base += 1
    
    return [count,locations]


In [9]:
#Test your code here
findmotifs(Seq('GATATATGCATATACTT'),Seq('ATAT')) #Should return [3, [2, 4, 10]]

[3, [2, 4, 10]]

Great! Sometimes, we often do not know exactly what motif we are looking for. A real-world motif application is combing through a series of genomes to identify common regions between them. In the next exercise, we will consider a simple case where we look for the longest common shared motif in a given readset, as a longer motif generally indicates a greater shared function. The input of the function will be a list of sequences, and the output should return a single sequence.

Here is an example scaffold for your code:
- Sort the reads from shortest to longest using the `sorted` function, with the key being the lengths of the reads
- Seperate the shortest sequence and the rest of the readset into two variables
- Initialise a blank string to store the motif
- Start a `for` loop (with index i) iterating over the bases in the shortest sequence
    - **Inside this for loop**
    - Initialise another for loop (with index j) iterating from the current base to the rest of the shortest sequence
    - **Inside this for loop**:
        - Set the stem (potential longest motif) as the ith base to the j+1th base in the shortest sequence
        - Initialse a `found` boolean variable as `False`
        - Initialise yet _another_ for loop iterating over the rest of the sequences
            - **Inside this for loop**
            - If the stem was in the sequence, set `found` as true
            - If not, set `found` as false and break the loop
        - If found is true and the length of the current stem is longer than the stored motif, replace teh current stored motif with the stem
- Return the motif sequence


In [5]:
#Exercise 4.2
def commonmotif(readset):
    #Your code here
    srt_reads = sorted(readset, key=len)
    shortest_seq = srt_reads[0]
    seqs = srt_reads[1:]
    motif = ''
    
    for i in range(len(shortest_seq)):
        for j in range(i, len(shortest_seq)):
            stem = shortest_seq[i:j+1]
            found = False
            for seq in seqs:
                if stem in seq:
                    found = True
                else:
                    found = False
                    break
            if found and len(stem) > len(motif):
                motif = stem
    
    return Seq(motif)

In [7]:
#Test your code here
commonmotif([Seq('GATTACA'), Seq('TAGACCA'), Seq('ATACA')]) #Should return TA

Seq('TA')

Now, we can try actually modifying a seqeunce, this time by removing adapter seqeunces. Adapter sequences are known artifical DNA sequences that serve to weed out specific DNA sequences of interest furing illumina sequencing, and are generally removed as they are not part of the sequence of interest. In the last exercise of this workshop, we will create a function that will take a seqeunce and motif of interest, and remove that motif from the original sequence, returning the modified sequence.

The function below takes four arguments: The readset, adapter sequence in question, the minimum match and the minimum length left over after removal. Output a list of sequences with the adapter sequence within them removed. If the sequence left over is lower than the minimum allowed length, remove the sequence from the readset.

In [2]:
#Exercise 4.3
def adapter_trim(reads, adapter_seq, min_match, min_length):
    #Your code here
    ### Get possible matches for adapter sequence
    possible_matches = []
    for i in range(len(adapter_seq) - min_match + 1):
        possible_matches.append(str(adapter_seq)[i:i + min_match])

    ### Trim adapter sequences
    trimmed_reads = []
    for iRead in reads:
        my_matches = [len(iRead)]
        
        # Check  all possible match indexes
        # before trimming read
        for iMatch in possible_matches:
            if iMatch in iRead:
                idx = str(iRead).find(str(iMatch))
                my_matches.append(idx)

        # Get the first match and trim
        # If no match, will return full read
        my_end = min(my_matches)
        trimmed_reads.append(iRead[0:my_end])

    ### Filter trimmed reads
    filtered_reads = []
    for iRead in trimmed_reads:
        
        # Remove any read that is min_length bases long or shorter
        # It was a common error to do if len(iRead) >= min_length
        if len(iRead) > min_length:
            filtered_reads.append(iRead)

    return filtered_reads

In [46]:
demo_reads = [Seq('GTTGGATTCATGAAAGA'), Seq('ATGAAATGAATGTCTTGA')]
demo_adapter = Seq('GATGAAAG')
print(adapter_trim(demo_reads, demo_adapter, 7, 15)) # should output a list with just one sequence: ATGAAATGAATGTCTTGA


[Seq('ATGAAATGAATGTCTTGA')]


Excellent! This is the end of workshop 1 of Bioinformatics. Thanks for attending, and see you next time for workshop 2, where we will look at seqeunce alignment!